In [ ]:
import sqlalchemy
import pandas as pd

import json

In [ ]:
file = 'resources/credential/Cloud_SQL.json'
with open(file, 'r') as f:
    credential = json.load(f)

In [ ]:
username = credential['username']
password = credential['password']
host = credential['host']
port = credential['port']
db = credential['database']

engine = sqlalchemy.create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{db}')

In [ ]:
# Sample Data
periods = [f'2019{x+1:02}' for x in range(3)]
chunk_size = 10000

for period in periods:
    currency_pair = 'AUDUSD'
    file = f'resources/data/DAT_ASCII_{currency_pair}_T_{period}.csv'
    print(f'Reading: {file}')
    
    df_chunks = pd.read_csv(file, sep=',',
                            header=None, names=['datetime', 'bid', 'ask', 'vol'],
                            usecols=['datetime', 'bid', 'ask'],
                            parse_dates=["datetime"],
                            date_parser=lambda x: pd.to_datetime(x, format="%Y%m%d %H%M%S%f"),
                            chunksize=chunk_size)
    
    full_df = pd.concat(df_chunks)
    full_df['currency_pair'] = currency_pair
    full_df['status'] = 'A'

In [ ]:
# Insert Data
full_df.to_sql(name='ticks', con=engine, index=False, if_exists='append',
               dtype={
                   'datetime': sqlalchemy.dialects.mysql.DATETIME(fsp=3),
                   'bid': sqlalchemy.DECIMAL(5, 5),
                   'ask': sqlalchemy.DECIMAL(5, 5),
                   'currency_pair': sqlalchemy.VARCHAR(10),
                   'status': sqlalchemy.Enum('A', 'M'),
               })

In [ ]:
# Select Data
sql = f'''
SELECT *
FROM ticks
'''

df_chunks = pd.read_sql_query(sql, engine, chunksize=10_000)
res_df = pd.concat(df_chunks)
res_df.head()